# Import package

In [1]:
import os
import time
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import random
from IPython import display
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [2]:
def set_seed(seed=200):
    tf.random.set_seed(seed)

    # optional
    # for numpy.random
    np.random.seed(seed)
    # for built-in random
    random.seed(seed)
    # for hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)

In [3]:
set_seed(5714)

# Testing paramemter setting

In [4]:
LAMBDA = 10

# Delet old checkpoint

!rm -rf /home/u7201623/mtsGAN_MSE/checkpoints/train/*

# Model construction

In [6]:
def make_mts_generator():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape = 1000))
    model.add(tf.keras.layers.Dense(512))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    
    model.add(tf.keras.layers.Dense(256))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    
    model.add(tf.keras.layers.Dense(128))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    
    model.add(tf.keras.layers.Dense(256))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())

    model.add(tf.keras.layers.Dense(512))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    
    model.add(tf.keras.layers.Dense(1000))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    return model

def make_stm_generator():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape = 1000))
    model.add(tf.keras.layers.Dense(512))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    
    model.add(tf.keras.layers.Dense(256))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    
    model.add(tf.keras.layers.Dense(128))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    
    model.add(tf.keras.layers.Dense(256))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())

    model.add(tf.keras.layers.Dense(512))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    
    model.add(tf.keras.layers.Dense(1000))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    
    return model

In [7]:
def make_micro_discriminator():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape = 1000))
    model.add(tf.keras.layers.Dense(512))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    
    model.add(tf.keras.layers.Dense(128))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    
    model.add(tf.keras.layers.Dense(32))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    
    model.add(tf.keras.layers.Dense(8))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    
    model.add(tf.keras.layers.Dense(1))

    
    return model

def make_scRNA_discriminator():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape = 1000))
    model.add(tf.keras.layers.Dense(512))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    
    model.add(tf.keras.layers.Dense(128))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    
    model.add(tf.keras.layers.Dense(32))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    
    model.add(tf.keras.layers.Dense(8))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    
    model.add(tf.keras.layers.Dense(1))
    
    return model

In [8]:
mts_generator = make_mts_generator()
stm_generator = make_stm_generator()
micro_discriminator = make_micro_discriminator()
scRNA_discriminator = make_scRNA_discriminator()

# Define the loss and optimizers

Define loss functions and optimizers for both models.

In [9]:
# This method returns a helper function to compute cross entropy loss
loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [10]:
def discriminator_loss(real, generated):

    real_loss = loss_obj(tf.ones_like(real), real)
    generated_loss = loss_obj(tf.zeros_like(generated), generated)
    total_disc_loss = real_loss + generated_loss
    
    return total_disc_loss * 0.5

In [11]:
def generator_loss(generated):
    return loss_obj(tf.ones_like(generated), generated)

In [12]:
def calc_cycle_loss(real_expr, cycled_expr):
    loss1 = tf.reduce_mean(tf.square(real_expr - cycled_expr))

    return LAMBDA * loss1

# Opitimizer setting

In [14]:
mts_generator_optimizer = tf.keras.optimizers.SGD(2e-4)
stm_generator_optimizer = tf.keras.optimizers.SGD(2e-4)
micro_discriminator_optimizer = tf.keras.optimizers.SGD(2e-5)
scRNA_discriminator_optimizer = tf.keras.optimizers.SGD(2e-5)

# Checkpoints

In [15]:
checkpoint_path = "./checkpoints/train/"

ckpt = tf.train.Checkpoint(mts_generator = mts_generator,
                           stm_generator = stm_generator,
                           micro_discriminator = micro_discriminator,
                           scRNA_discriminator = scRNA_discriminator,
                           mts_generator_optimizer = mts_generator_optimizer,
                           stm_generator_optimizer = stm_generator_optimizer,
                           micro_discriminator_optimizer = micro_discriminator_optimizer,
                           scRNA_discriminator_optimizer = scRNA_discriminator_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep = 2)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')

Latest checkpoint restored!!


# Import real scRNA exprs and real microarray exprs

In [16]:
scRNA = pd.read_hdf("./15.Train.h5")
test_array = pd.read_csv("./3.Real_micro.csv", index_col = 0)

In [17]:
test_array

,200602_at,200665_s_at,200696_s_at,200771_at,200783_s_at,200796_s_at,200832_s_at,200850_s_at,200862_at,200878_at,...,37793_r_at,38241_at,39248_at,41644_at,47553_at,49306_at,49452_at,52285_f_at,58780_s_at,87100_at
GSM2477208,4.911315,4.131889,5.271507,3.595087,5.420753,9.186289,4.536803,5.777385,4.046164,3.913686,...,5.377377,7.871929,11.588005,4.208405,5.132027,3.733284,8.336109,4.354907,3.636091,8.007895
GSM2477211,6.300982,4.574658,5.066376,3.107291,4.792333,10.087429,4.590755,4.696675,4.219169,3.726054,...,4.740422,9.055924,11.563269,4.429010,4.581212,3.714408,8.263204,2.944234,5.054619,8.183984
GSM2477214,6.014036,4.361766,4.546221,3.056467,6.593299,9.031646,4.494644,6.788082,5.359601,6.224937,...,4.071003,8.508700,11.920443,4.410648,4.648817,4.223868,9.933782,3.998677,5.081192,8.743885
GSM2477217,7.340784,4.801688,4.735673,3.133222,6.603018,8.653825,4.776246,5.128331,4.345273,4.884019,...,6.758454,8.095148,11.691123,4.580441,4.217653,3.597429,10.725069,5.491937,3.793091,8.562646
GSM2477220,7.241149,3.777834,4.778465,3.256823,6.035418,9.660932,4.157027,6.625359,4.489071,3.756436,...,4.010713,9.291461,11.793123,4.625945,6.542801,3.839529,8.609015,5.049157,3.483192,5.894935
GSM2477223,6.871384,3.852978,4.642586,2.771044,5.861102,9.370401,5.556481,5.498355,4.250759,5.596937,...,5.548992,8.642527,12.018413,4.467523,4.577796,4.132177,8.629219,5.330513,3.860879,8.264344
GSM2477226,5.728606,4.294980,4.675356,3.440598,5.543965,10.002856,4.406171,5.588373,3.861911,4.257667,...,4.769680,8.149686,11.319660,4.115787,4.596850,3.301257,8.906163,4.361480,4.126293,8.438564
GSM2477229,5.590641,4.174627,4.489352,4.040324,5.184063,9.529588,5.289741,6.410958,3.723480,3.780705,...,3.977643,8.796014,11.618819,4.184158,4.399476,3.332777,7.680005,3.224442,4.006135,7.028182
GSM2477232,6.689630,4.796120,5.158561,3.234776,5.434718,8.635900,4.523756,5.056871,3.678943,4.163047,...,4.433314,8.605020,11.615741,5.009605,4.302277,3.711505,9.841388,3.970889,4.509085,8.074000
GSM2477235,5.636394,4.235991,4.612142,3.170471,5.816085,9.674786,6.441321,7.097888,3.928385,4.418368,...,3.755050,8.667233,11.912315,4.529763,4.603975,3.812394,8.964866,4.641343,3.601624,8.462632


In [18]:
scRNA = scRNA.transpose()

In [20]:
Cell_barcode = pd.read_csv("0.test_cell.csv")
Cell_barcode = Cell_barcode.drop(["Unnamed: 0"], axis = 1)
Cell_barcode = Cell_barcode.rename(columns={"1":"barcode", "Unnamed: 2": "cell type"})
Cell_barcode

,barcode,cell type
0,AGCTTTACAAGGGC-1_4,Monocyte
1,AGACGTACCCATGA-1_4,Monocyte
2,TGTGACGATTTGTC-1_4,Monocyte
3,TTAGGGTGCGGAGA-1_4,Monocyte
4,GCAACTGATGCCCT-1_4,Monocyte
...,...,...
78,ACGATTCTAGAATG-1_8,Naive_T
79,AGTTCTTGACGGGA-1_8,Naive_T
80,AGGACACTCTCGCT-1_8,Naive_T
81,TGGACTGACCCGTT-1_8,Naive_T


In [21]:
test_cell = scRNA.loc[Cell_barcode["barcode"]]

In [22]:
test_cell 

,ENSG00000000938,ENSG00000002549,ENSG00000002586,ENSG00000005022,ENSG00000007264,ENSG00000007312,ENSG00000008018,ENSG00000008282,ENSG00000008517,ENSG00000008988,...,ENSG00000244754,ENSG00000245532,ENSG00000250479,ENSG00000251562,ENSG00000254999,ENSG00000255112,ENSG00000255302,ENSG00000265681,ENSG00000269893,ENSG00000271614
AGCTTTACAAGGGC-1_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,6.0,0.0,0.0,1.0,0.0,0.0,0.0
AGACGTACCCATGA-1_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0
TGTGACGATTTGTC-1_4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,1.0,0.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0
TTAGGGTGCGGAGA-1_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,...,1.0,3.0,0.0,11.0,0.0,0.0,0.0,2.0,0.0,1.0
GCAACTGATGCCCT-1_4,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,19.0,2.0,2.0,0.0,1.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACGATTCTAGAATG-1_8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,...,0.0,0.0,0.0,22.0,0.0,0.0,0.0,1.0,0.0,0.0
AGTTCTTGACGGGA-1_8,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,6.0,...,0.0,0.0,0.0,39.0,0.0,0.0,0.0,1.0,0.0,0.0
AGGACACTCTCGCT-1_8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,1.0,45.0,1.0,1.0,0.0,3.0,0.0,0.0
TGGACTGACCCGTT-1_8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,...,0.0,0.0,0.0,30.0,0.0,0.0,0.0,2.0,0.0,0.0


In [23]:
test_cell.to_csv("./1.Real_scRNA.csv")

In [24]:
test_scRNA = np.array(test_cell)

In [25]:
test_scRNA.shape

(83, 1000)

In [26]:
test_cell

,ENSG00000000938,ENSG00000002549,ENSG00000002586,ENSG00000005022,ENSG00000007264,ENSG00000007312,ENSG00000008018,ENSG00000008282,ENSG00000008517,ENSG00000008988,...,ENSG00000244754,ENSG00000245532,ENSG00000250479,ENSG00000251562,ENSG00000254999,ENSG00000255112,ENSG00000255302,ENSG00000265681,ENSG00000269893,ENSG00000271614
AGCTTTACAAGGGC-1_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,6.0,0.0,0.0,1.0,0.0,0.0,0.0
AGACGTACCCATGA-1_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0
TGTGACGATTTGTC-1_4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,1.0,0.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0
TTAGGGTGCGGAGA-1_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,...,1.0,3.0,0.0,11.0,0.0,0.0,0.0,2.0,0.0,1.0
GCAACTGATGCCCT-1_4,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,19.0,2.0,2.0,0.0,1.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACGATTCTAGAATG-1_8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,...,0.0,0.0,0.0,22.0,0.0,0.0,0.0,1.0,0.0,0.0
AGTTCTTGACGGGA-1_8,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,6.0,...,0.0,0.0,0.0,39.0,0.0,0.0,0.0,1.0,0.0,0.0
AGGACACTCTCGCT-1_8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,1.0,45.0,1.0,1.0,0.0,3.0,0.0,0.0
TGGACTGACCCGTT-1_8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,...,0.0,0.0,0.0,30.0,0.0,0.0,0.0,2.0,0.0,0.0


In [27]:
def rename_table(place, target_table, reference_table):
    if place == "column":
        col_dict = {}
        for i in range(len(reference_table.columns)):
            col_dict[i] = str(reference_table.columns[i])
        target_table = target_table.rename(columns = col_dict)
        return target_table
    if place == "index":
        index_dict = {}
        for i in range(len(reference_table.index)):
            index_dict[i] = str(reference_table.index[i])
        target_table = target_table.rename(index = index_dict)
        return target_table

# Fake micro

In [28]:
fake_micro = stm_generator(test_scRNA.reshape(83, 1000), training = False)
fake_micro.shape

TensorShape([83, 1000])

In [29]:
np_fake_micro = fake_micro.numpy()
fake_micro_table = pd.DataFrame(np_fake_micro)

In [30]:
fake_micro_table

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,7.594425,3.619957,3.568396,5.386302,3.333866,0.000000,4.543201,1.519645,10.894630,0.000000,...,0.475803,8.255483,15.359475,11.379560,5.000095,7.319702,8.549360,6.588502,6.280713,5.368527
1,10.552801,1.185442,0.000000,5.132748,1.973832,0.000000,5.108559,0.000000,12.582154,0.000000,...,0.867625,10.893318,17.301842,14.201316,6.386200,10.171132,9.330461,9.022736,5.786565,0.958376
2,6.763441,4.977327,5.791384,5.617873,3.346910,1.473713,5.890306,2.770046,10.640773,0.109905,...,1.006107,7.327617,14.374288,10.165351,5.006154,5.761745,7.848447,5.586961,6.907032,7.011469
3,8.036524,3.610886,3.250960,5.306488,3.550255,0.000000,5.540271,1.567289,11.534882,0.000000,...,0.952353,8.803999,14.544120,12.024785,5.488193,8.465459,8.630926,7.594312,6.079993,4.607288
4,4.511441,5.871610,8.332629,5.356959,4.961732,4.504587,7.513024,6.329903,7.523500,4.985375,...,1.163705,6.007492,10.577840,6.709049,3.144874,2.374186,8.143645,3.571703,5.163762,9.659129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,3.866081,4.101872,5.452798,4.959390,6.356052,6.497402,6.779802,8.736065,5.000596,6.530637,...,6.402422,5.393868,7.223303,4.662335,1.461111,3.104501,7.467319,4.395817,2.216506,7.492093
79,3.826801,3.066331,5.725656,3.673255,8.072450,6.172884,5.120073,8.016782,3.729273,5.568055,...,5.500967,5.731333,5.256433,4.780551,3.406316,4.865752,7.425111,4.830995,3.769316,5.266829
80,3.659122,3.751928,6.076849,4.488760,6.077988,6.287530,7.729692,8.607069,5.019086,5.918157,...,6.348950,5.668190,6.369729,4.740458,2.133152,3.022396,7.068439,4.569431,3.620188,6.344712
81,5.555356,2.660467,4.941579,4.132542,6.508596,4.111393,4.403243,7.756066,4.376408,5.595559,...,4.765658,6.935573,5.975076,4.402139,2.704373,4.668320,7.016990,3.964967,1.776004,4.899920


In [31]:
fake_micro_table = rename_table("column", fake_micro_table, test_array)
fake_micro_table = rename_table("index", fake_micro_table, test_cell)

In [32]:
fake_micro_table

,200602_at,200665_s_at,200696_s_at,200771_at,200783_s_at,200796_s_at,200832_s_at,200850_s_at,200862_at,200878_at,...,37793_r_at,38241_at,39248_at,41644_at,47553_at,49306_at,49452_at,52285_f_at,58780_s_at,87100_at
AGCTTTACAAGGGC-1_4,7.594425,3.619957,3.568396,5.386302,3.333866,0.000000,4.543201,1.519645,10.894630,0.000000,...,0.475803,8.255483,15.359475,11.379560,5.000095,7.319702,8.549360,6.588502,6.280713,5.368527
AGACGTACCCATGA-1_4,10.552801,1.185442,0.000000,5.132748,1.973832,0.000000,5.108559,0.000000,12.582154,0.000000,...,0.867625,10.893318,17.301842,14.201316,6.386200,10.171132,9.330461,9.022736,5.786565,0.958376
TGTGACGATTTGTC-1_4,6.763441,4.977327,5.791384,5.617873,3.346910,1.473713,5.890306,2.770046,10.640773,0.109905,...,1.006107,7.327617,14.374288,10.165351,5.006154,5.761745,7.848447,5.586961,6.907032,7.011469
TTAGGGTGCGGAGA-1_4,8.036524,3.610886,3.250960,5.306488,3.550255,0.000000,5.540271,1.567289,11.534882,0.000000,...,0.952353,8.803999,14.544120,12.024785,5.488193,8.465459,8.630926,7.594312,6.079993,4.607288
GCAACTGATGCCCT-1_4,4.511441,5.871610,8.332629,5.356959,4.961732,4.504587,7.513024,6.329903,7.523500,4.985375,...,1.163705,6.007492,10.577840,6.709049,3.144874,2.374186,8.143645,3.571703,5.163762,9.659129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACGATTCTAGAATG-1_8,3.866081,4.101872,5.452798,4.959390,6.356052,6.497402,6.779802,8.736065,5.000596,6.530637,...,6.402422,5.393868,7.223303,4.662335,1.461111,3.104501,7.467319,4.395817,2.216506,7.492093
AGTTCTTGACGGGA-1_8,3.826801,3.066331,5.725656,3.673255,8.072450,6.172884,5.120073,8.016782,3.729273,5.568055,...,5.500967,5.731333,5.256433,4.780551,3.406316,4.865752,7.425111,4.830995,3.769316,5.266829
AGGACACTCTCGCT-1_8,3.659122,3.751928,6.076849,4.488760,6.077988,6.287530,7.729692,8.607069,5.019086,5.918157,...,6.348950,5.668190,6.369729,4.740458,2.133152,3.022396,7.068439,4.569431,3.620188,6.344712
TGGACTGACCCGTT-1_8,5.555356,2.660467,4.941579,4.132542,6.508596,4.111393,4.403243,7.756066,4.376408,5.595559,...,4.765658,6.935573,5.975076,4.402139,2.704373,4.668320,7.016990,3.964967,1.776004,4.899920


In [33]:
fake_micro_table.to_csv("2.Fake_micro.csv")

# Fake scRNA

In [34]:
test_micro = np.array(test_array)
test_micro.shape

(54, 1000)

In [35]:
fake_scRNA = mts_generator(test_micro.reshape(54, 1000), training = False)

In [36]:
fake_scRNA.shape
np_fake_scRNA = fake_scRNA.numpy()
fake_scRNA_table = pd.DataFrame(np_fake_scRNA)

In [37]:
fake_scRNA_table

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.000000,0.050498,0.000000,5.256750,0.438387,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.511869,1.465527,0.000000,81.410896,3.433257,0.000000,0.000000,6.999103,0.706236,0.000000
1,0.000000,0.283619,0.000000,5.793855,0.518028,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.730152,0.933004,0.000000,84.117760,4.620743,0.000000,0.000000,9.010546,1.265615,0.000000
2,0.000000,0.221995,0.075010,1.970637,0.156361,0.559730,0.307072,0.000000,0.001602,2.111867,...,0.000000,2.141412,0.000000,77.683464,0.000000,0.000000,0.000000,0.370709,0.000000,0.000000
3,0.000000,0.428776,0.000000,1.026777,0.191820,0.828668,0.901531,0.000000,1.880987,2.841088,...,0.000000,3.182949,0.000000,75.173744,0.000000,0.000000,0.193578,0.716132,0.029274,0.000000
4,0.000000,0.365126,0.044490,1.354805,0.164636,0.829231,0.682671,0.000000,1.494171,2.841288,...,0.000000,2.422772,0.000000,76.368057,0.000000,0.000000,0.242598,0.476154,0.000000,0.000000
5,0.000000,0.393685,0.000000,1.451277,0.276293,0.507648,0.750920,0.000000,1.131598,2.839266,...,0.000000,2.300518,0.000000,75.267380,0.000000,0.000000,0.024794,0.699189,0.000000,0.000000
6,0.000000,0.126412,0.000000,4.533046,0.381789,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.145683,2.257291,0.000000,79.982231,2.353713,0.000000,0.000000,5.337173,0.404214,0.000000
7,0.000000,0.201905,0.000000,4.072022,0.497971,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.799163,2.328969,0.000000,77.444191,1.680954,0.000000,0.000000,4.405495,0.219602,0.000000
8,0.000000,0.208317,0.000000,2.665551,0.474606,0.142852,0.000000,0.000000,0.000000,1.361532,...,0.227156,2.171285,0.000000,76.238586,0.000000,0.000000,0.000000,1.287771,0.000000,0.000000
9,0.000000,0.103114,0.000000,5.532369,0.423692,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.683008,0.923202,0.000000,80.489708,4.008238,0.000000,0.000000,8.125850,1.087983,0.000000


In [38]:
fake_scRNA_table = rename_table("column", fake_scRNA_table, test_cell)
fake_scRNA_table = rename_table("index", fake_scRNA_table, test_array)

In [39]:
fake_scRNA_table

,ENSG00000000938,ENSG00000002549,ENSG00000002586,ENSG00000005022,ENSG00000007264,ENSG00000007312,ENSG00000008018,ENSG00000008282,ENSG00000008517,ENSG00000008988,...,ENSG00000244754,ENSG00000245532,ENSG00000250479,ENSG00000251562,ENSG00000254999,ENSG00000255112,ENSG00000255302,ENSG00000265681,ENSG00000269893,ENSG00000271614
GSM2477208,0.000000,0.050498,0.000000,5.256750,0.438387,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.511869,1.465527,0.000000,81.410896,3.433257,0.000000,0.000000,6.999103,0.706236,0.000000
GSM2477211,0.000000,0.283619,0.000000,5.793855,0.518028,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.730152,0.933004,0.000000,84.117760,4.620743,0.000000,0.000000,9.010546,1.265615,0.000000
GSM2477214,0.000000,0.221995,0.075010,1.970637,0.156361,0.559730,0.307072,0.000000,0.001602,2.111867,...,0.000000,2.141412,0.000000,77.683464,0.000000,0.000000,0.000000,0.370709,0.000000,0.000000
GSM2477217,0.000000,0.428776,0.000000,1.026777,0.191820,0.828668,0.901531,0.000000,1.880987,2.841088,...,0.000000,3.182949,0.000000,75.173744,0.000000,0.000000,0.193578,0.716132,0.029274,0.000000
GSM2477220,0.000000,0.365126,0.044490,1.354805,0.164636,0.829231,0.682671,0.000000,1.494171,2.841288,...,0.000000,2.422772,0.000000,76.368057,0.000000,0.000000,0.242598,0.476154,0.000000,0.000000
GSM2477223,0.000000,0.393685,0.000000,1.451277,0.276293,0.507648,0.750920,0.000000,1.131598,2.839266,...,0.000000,2.300518,0.000000,75.267380,0.000000,0.000000,0.024794,0.699189,0.000000,0.000000
GSM2477226,0.000000,0.126412,0.000000,4.533046,0.381789,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.145683,2.257291,0.000000,79.982231,2.353713,0.000000,0.000000,5.337173,0.404214,0.000000
GSM2477229,0.000000,0.201905,0.000000,4.072022,0.497971,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.799163,2.328969,0.000000,77.444191,1.680954,0.000000,0.000000,4.405495,0.219602,0.000000
GSM2477232,0.000000,0.208317,0.000000,2.665551,0.474606,0.142852,0.000000,0.000000,0.000000,1.361532,...,0.227156,2.171285,0.000000,76.238586,0.000000,0.000000,0.000000,1.287771,0.000000,0.000000
GSM2477235,0.000000,0.103114,0.000000,5.532369,0.423692,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.683008,0.923202,0.000000,80.489708,4.008238,0.000000,0.000000,8.125850,1.087983,0.000000


In [40]:
fake_scRNA_table.to_csv("4.Fake_scRNA.csv")

# Cycle microarray

In [41]:
cycle_micro = stm_generator(np_fake_scRNA.reshape(54, 1000), training = False)

In [42]:
np_cycle_micro = cycle_micro.numpy()
cycle_micro_table = pd.DataFrame(np_cycle_micro)

In [43]:
cycle_micro_table

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,3.906485,3.667176,5.223503,3.444797,7.089633,5.595369,3.910221,5.662729,4.226273,3.139912,...,7.783350,8.749398,10.002807,4.809225,2.579033,3.984613,5.712173,2.852515,4.159746,6.348783
1,2.205057,3.203548,3.164956,1.842735,7.594975,5.364645,0.533650,2.783875,3.072538,1.440770,...,8.673541,10.118031,9.722374,3.623979,2.053025,5.419729,5.129533,0.124561,3.556504,2.134499
2,4.857671,3.220389,5.159583,3.375427,6.451070,6.191699,4.733238,6.595146,4.860086,3.796949,...,5.416362,8.134041,10.001255,4.573494,4.278657,3.589977,7.402183,3.814736,4.555374,6.866060
3,5.130860,3.441451,5.265688,3.280915,6.481351,6.171008,4.878170,6.840898,4.767103,3.814847,...,5.634297,8.258413,10.140231,4.717104,4.514417,3.353349,7.531936,4.181401,4.582788,6.498714
4,5.156673,3.135298,5.191178,3.299824,6.449117,6.473453,4.745953,6.739556,4.880190,3.818951,...,5.407025,8.048701,9.969183,4.638322,4.572150,3.694674,7.439259,3.916464,4.567607,6.690520
5,5.027274,3.314457,5.168811,3.332366,6.448390,6.197331,4.720639,6.724000,4.797519,3.873966,...,5.465065,8.164140,10.088327,4.682589,4.548750,3.522025,7.383248,4.015935,4.691079,6.612179
6,4.430154,3.463841,5.570727,3.997277,6.801385,5.890276,4.620229,6.413711,4.457380,3.396453,...,6.618464,7.874589,10.043132,4.820807,3.007012,3.663863,6.131193,3.455789,4.668823,7.128525
7,4.476024,3.400524,5.523047,4.016094,6.575897,5.615151,4.565693,6.532771,4.479296,3.537358,...,6.362790,7.503662,9.921825,4.654790,2.995209,3.631506,6.341455,3.468393,4.651920,7.129441
8,4.789055,3.146323,5.166354,3.529649,6.396678,5.895180,4.632510,6.455143,4.662861,3.773499,...,5.850381,7.864997,9.829515,4.539745,3.658197,3.466773,7.302855,3.797121,4.517421,6.833145
9,3.575062,3.272877,4.542309,3.034585,7.232181,5.722456,2.557181,4.773325,4.085464,2.896922,...,8.036704,9.262237,10.070673,4.279783,2.218950,4.819122,5.667190,1.950627,4.002553,5.089820


In [44]:
cycle_micro_table = rename_table("column", cycle_micro_table, test_array)
cycle_micro_table = rename_table("index", cycle_micro_table, test_array)

In [45]:
cycle_micro_table

,200602_at,200665_s_at,200696_s_at,200771_at,200783_s_at,200796_s_at,200832_s_at,200850_s_at,200862_at,200878_at,...,37793_r_at,38241_at,39248_at,41644_at,47553_at,49306_at,49452_at,52285_f_at,58780_s_at,87100_at
GSM2477208,3.906485,3.667176,5.223503,3.444797,7.089633,5.595369,3.910221,5.662729,4.226273,3.139912,...,7.783350,8.749398,10.002807,4.809225,2.579033,3.984613,5.712173,2.852515,4.159746,6.348783
GSM2477211,2.205057,3.203548,3.164956,1.842735,7.594975,5.364645,0.533650,2.783875,3.072538,1.440770,...,8.673541,10.118031,9.722374,3.623979,2.053025,5.419729,5.129533,0.124561,3.556504,2.134499
GSM2477214,4.857671,3.220389,5.159583,3.375427,6.451070,6.191699,4.733238,6.595146,4.860086,3.796949,...,5.416362,8.134041,10.001255,4.573494,4.278657,3.589977,7.402183,3.814736,4.555374,6.866060
GSM2477217,5.130860,3.441451,5.265688,3.280915,6.481351,6.171008,4.878170,6.840898,4.767103,3.814847,...,5.634297,8.258413,10.140231,4.717104,4.514417,3.353349,7.531936,4.181401,4.582788,6.498714
GSM2477220,5.156673,3.135298,5.191178,3.299824,6.449117,6.473453,4.745953,6.739556,4.880190,3.818951,...,5.407025,8.048701,9.969183,4.638322,4.572150,3.694674,7.439259,3.916464,4.567607,6.690520
GSM2477223,5.027274,3.314457,5.168811,3.332366,6.448390,6.197331,4.720639,6.724000,4.797519,3.873966,...,5.465065,8.164140,10.088327,4.682589,4.548750,3.522025,7.383248,4.015935,4.691079,6.612179
GSM2477226,4.430154,3.463841,5.570727,3.997277,6.801385,5.890276,4.620229,6.413711,4.457380,3.396453,...,6.618464,7.874589,10.043132,4.820807,3.007012,3.663863,6.131193,3.455789,4.668823,7.128525
GSM2477229,4.476024,3.400524,5.523047,4.016094,6.575897,5.615151,4.565693,6.532771,4.479296,3.537358,...,6.362790,7.503662,9.921825,4.654790,2.995209,3.631506,6.341455,3.468393,4.651920,7.129441
GSM2477232,4.789055,3.146323,5.166354,3.529649,6.396678,5.895180,4.632510,6.455143,4.662861,3.773499,...,5.850381,7.864997,9.829515,4.539745,3.658197,3.466773,7.302855,3.797121,4.517421,6.833145
GSM2477235,3.575062,3.272877,4.542309,3.034585,7.232181,5.722456,2.557181,4.773325,4.085464,2.896922,...,8.036704,9.262237,10.070673,4.279783,2.218950,4.819122,5.667190,1.950627,4.002553,5.089820


In [46]:
cycle_micro_table.to_csv("5.Cycle_micro.csv")

# Cycle scRNA

In [47]:
cycle_scRNA = mts_generator(np_fake_micro.reshape(83, 1000), training = False)

In [48]:
np_cycle_scRNA = cycle_scRNA.numpy()
cycle_scRNA_table = pd.DataFrame(np_cycle_scRNA)

In [49]:
cycle_scRNA_table = rename_table("column", cycle_scRNA_table, test_cell)
cycle_scRNA_table = rename_table("index", cycle_scRNA_table, test_cell)

In [50]:
cycle_scRNA_table

,ENSG00000000938,ENSG00000002549,ENSG00000002586,ENSG00000005022,ENSG00000007264,ENSG00000007312,ENSG00000008018,ENSG00000008282,ENSG00000008517,ENSG00000008988,...,ENSG00000244754,ENSG00000245532,ENSG00000250479,ENSG00000251562,ENSG00000254999,ENSG00000255112,ENSG00000255302,ENSG00000265681,ENSG00000269893,ENSG00000271614
AGCTTTACAAGGGC-1_4,0.152119,0.0,0.000000,0.074607,0.000000,0.218949,0.000000,0.000000,1.855608,4.947573,...,0.000000,0.366601,0.0,40.893486,0.000000,0.000000,0.0,1.524386,0.000000,0.460130
AGACGTACCCATGA-1_4,0.144032,0.0,0.000000,0.000000,0.000000,1.135426,0.000000,0.000000,2.301936,7.265987,...,0.000000,1.601040,0.0,26.871222,0.259541,0.000000,0.0,1.537900,0.132255,0.380283
TGTGACGATTTGTC-1_4,0.032375,0.0,0.000000,0.183188,0.000000,0.000000,0.000000,0.000000,1.579352,5.362389,...,0.000000,0.538234,0.0,43.686287,0.000000,0.000000,0.0,1.392136,0.000000,0.371830
TTAGGGTGCGGAGA-1_4,0.050763,0.0,0.000000,0.000000,0.000000,0.587486,0.000000,0.000000,3.186749,6.759381,...,0.000000,1.788078,0.0,38.865833,0.097293,0.000000,0.0,1.524112,0.000000,0.409491
GCAACTGATGCCCT-1_4,0.000000,0.0,0.000000,0.424340,0.000000,0.000000,0.234941,0.000000,1.004758,4.050174,...,0.000000,0.408176,0.0,46.772892,0.543332,0.000000,0.0,2.150810,0.000000,0.210841
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACGATTCTAGAATG-1_8,0.119755,0.0,0.000000,0.553101,0.257100,0.000000,0.042711,0.000000,1.182922,3.978875,...,0.281769,0.357040,0.0,43.354980,0.021109,0.338698,0.0,2.307979,0.000000,0.000000
AGTTCTTGACGGGA-1_8,0.141173,0.0,0.109469,0.247962,0.417374,0.000000,0.246099,0.000000,0.736954,5.700101,...,0.242009,0.387795,0.0,42.669754,0.000000,0.178215,0.0,2.123127,0.000000,0.000000
AGGACACTCTCGCT-1_8,0.174743,0.0,0.000000,0.331900,0.277387,0.000000,0.316184,0.000000,0.118666,4.010135,...,0.105974,0.011192,0.0,48.643665,0.000000,0.409331,0.0,2.726500,0.000000,0.000000
TGGACTGACCCGTT-1_8,0.000000,0.0,0.000000,0.639342,0.422117,0.000000,0.247950,0.119312,1.742262,4.381091,...,0.000000,0.037325,0.0,42.800838,0.000000,0.385801,0.0,2.258895,0.000000,0.000000


In [51]:
cycle_scRNA_table.to_csv("6.Cycle_scRNA.csv")